In [1]:
import numpy as np
import random as rand

In [2]:
num_train_points = 100
num_test_points = 1000
learning_rate = 0.01

In [3]:
def generate_target_function():
    x0 = rand.uniform(-1, 1)
    y0 = rand.uniform(-1, 1)
    x1 = rand.uniform(-1, 1)
    y1 = rand.uniform(-1, 1)

    m_f = (y1 - y0) / (x1 - x0)
    b_f = y0 - m_f * x0
    f = [m_f, b_f]
    
    return f

In [4]:
def classify_point(m, b, x, y):
    expected_value = m * x + b
    
    if y >= expected_value:
        return 1
    else:
        return -1

In [5]:
def generate_data(m, b, num_points):
    bias = [1 for _ in range(num_points)]
    x1 = [rand.uniform(-1, 1) for _ in range(num_points)]
    x2 = [rand.uniform(-1, 1) for _ in range(num_points)]

    xn_pre_bias = np.column_stack((x1, x2))
    xn = np.column_stack((bias, xn_pre_bias))
    yn = [classify_point(m, b, x1, x2) for (bias, x1, x2) in xn]

    training_data = np.column_stack((xn, yn))
    
    return training_data

In [6]:
def norm(vec):
    return np.sqrt(np.sum(vec * vec))

In [7]:
def SGD(input_weight, training_data):
    perm_training_data = np.random.permutation(training_data)
    xn = np.transpose(np.delete(np.transpose(perm_training_data), 3, 0))
    yn = np.transpose(np.transpose(perm_training_data)[3])

    N = len(xn)

    for i in range(N):
        xi = xn[i]
        yi = yn[i]
        
        gradient = -1 * (yi * xi) / (1 + np.exp(yi * np.dot(input_weight, xi)))
        input_weight = input_weight - learning_rate * gradient
    
    return input_weight

In [8]:
def calc_cross_entropy_error(testing_data, weight):
    xn = np.transpose(np.delete(np.transpose(testing_data), 3, 0))
    yn = np.transpose(np.transpose(testing_data)[3])
    
    N = len(xn)
    
    error = 0
    
    for i in range(N):
        xi = xn[i]
        yi = yn[i]
        
        error += np.log(1 + np.exp(-yi * np.dot(weight, xi)))
    
    return (1 / N) * error

In [9]:
Eout_errors = []
epoch_weights = np.array([[0, 0, 0]])
num_epochs_arr = []

In [12]:
def run_experiment():
    m_f, b_f = generate_target_function()
    # generate target function
    train_data = generate_data(m_f, b_f, num_train_points)

    # run SGD
        # run N epochs till subsequent weights are within two decimal places
    
    prev_weight = epoch_weights[-1]
    weight = SGD(prev_weight, train_data)
    epoch_iter = 1
    
    while norm(prev_weight - weight) >= 0.01:
        np.append(epoch_weights, [weight], axis=0)
        prev_weight = weight
        weight = SGD(prev_weight, train_data)
        epoch_iter += 1

    num_epochs_arr.append(epoch_iter)
    # generate testing data
    testing_data = generate_data(m_f, b_f, num_test_points)
    # estimate E_out
    Eout_errors.append(calc_cross_entropy_error(testing_data, weight))

In [13]:
for i in range(100):
    run_experiment()

In [14]:
np.mean(Eout_errors)

0.10069016921792245

In [15]:
np.mean(num_epochs_arr)

335.5